# CEBD 1261 Winter 2020
## Final Project: Mushroom classification (Poisonous (p) vs. Edible (e))
### Data source: https://www.kaggle.com/uciml/mushroom-classification 
### By: Pawel Kaluski


Searching for data to use for my project I found this one. It is a classification problem. The challenges with this dataset are that it only has characters and no numbers. It requires alot of encoding. The column names use a '-' to separate words. This creates issues.

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql as sparksql
spark = SparkSession.builder.appName('mushrooms').getOrCreate()
train = spark.read.csv('mushrooms.csv', inferSchema=True,header=True)
import pandas as pd

In [2]:
# create DataFrame as a temporary view
train.createOrReplaceTempView('table')

In [3]:
train.printSchema()

root
 |-- class: string (nullable = true)
 |-- cap-shape: string (nullable = true)
 |-- cap-surface: string (nullable = true)
 |-- cap-color: string (nullable = true)
 |-- bruises: string (nullable = true)
 |-- odor: string (nullable = true)
 |-- gill-attachment: string (nullable = true)
 |-- gill-spacing: string (nullable = true)
 |-- gill-size: string (nullable = true)
 |-- gill-color: string (nullable = true)
 |-- stalk-shape: string (nullable = true)
 |-- stalk-root: string (nullable = true)
 |-- stalk-surface-above-ring: string (nullable = true)
 |-- stalk-surface-below-ring: string (nullable = true)
 |-- stalk-color-above-ring: string (nullable = true)
 |-- stalk-color-below-ring: string (nullable = true)
 |-- veil-type: string (nullable = true)
 |-- veil-color: string (nullable = true)
 |-- ring-number: string (nullable = true)
 |-- ring-type: string (nullable = true)
 |-- spore-print-color: string (nullable = true)
 |-- population: string (nullable = true)
 |-- habitat: string 

## Next we will look at te different features to determine what's in them

In [4]:
# Our Target
train.groupBy('class').count().show()

+-----+-----+
|class|count|
+-----+-----+
|    e| 4208|
|    p| 3916|
+-----+-----+



In [5]:
train.groupBy('cap-shape').count().show()

+---------+-----+
|cap-shape|count|
+---------+-----+
|        x| 3656|
|        f| 3152|
|        k|  828|
|        c|    4|
|        b|  452|
|        s|   32|
+---------+-----+



In [6]:
train.groupBy('cap-surface').count().show()

+-----------+-----+
|cap-surface|count|
+-----------+-----+
|          g|    4|
|          f| 2320|
|          y| 3244|
|          s| 2556|
+-----------+-----+



In [7]:
train.groupBy('cap-color').count().show()

+---------+-----+
|cap-color|count|
+---------+-----+
|        g| 1840|
|        n| 2284|
|        e| 1500|
|        p|  144|
|        y| 1072|
|        w| 1040|
|        c|   44|
|        u|   16|
|        b|  168|
|        r|   16|
+---------+-----+



In [8]:
train.groupBy('bruises').count().show()

+-------+-----+
|bruises|count|
+-------+-----+
|      f| 4748|
|      t| 3376|
+-------+-----+



In [9]:
train.groupBy('odor').count().show()

+----+-----+
|odor|count|
+----+-----+
|   l|  400|
|   m|   36|
|   f| 2160|
|   n| 3528|
|   p|  256|
|   y|  576|
|   c|  192|
|   a|  400|
|   s|  576|
+----+-----+



In [10]:
train.groupBy('gill-attachment').count().show()

+---------------+-----+
|gill-attachment|count|
+---------------+-----+
|              f| 7914|
|              a|  210|
+---------------+-----+



In [11]:
train.groupBy('gill-spacing').count().show()

+------------+-----+
|gill-spacing|count|
+------------+-----+
|           w| 1312|
|           c| 6812|
+------------+-----+



In [12]:
train.groupBy('gill-size').count().show()

+---------+-----+
|gill-size|count|
+---------+-----+
|        n| 2512|
|        b| 5612|
+---------+-----+



In [13]:
train.groupBy('gill-color').count().show()

+----------+-----+
|gill-color|count|
+----------+-----+
|         g|  752|
|         n| 1048|
|         k|  408|
|         e|   96|
|         o|   64|
|         h|  732|
|         p| 1492|
|         w| 1202|
|         y|   86|
|         u|  492|
|         b| 1728|
|         r|   24|
+----------+-----+



In [14]:
train.groupBy('stalk-shape').count().show()

+-----------+-----+
|stalk-shape|count|
+-----------+-----+
|          e| 3516|
|          t| 4608|
+-----------+-----+



In [15]:
train.groupBy('stalk-root').count().show()

+----------+-----+
|stalk-root|count|
+----------+-----+
|         e| 1120|
|         c|  556|
|         b| 3776|
|         r|  192|
|         ?| 2480|
+----------+-----+



#### We can see we have 2480 missing values we can exclude this column in the MVP

In [16]:
train.groupBy('stalk-surface-above-ring').count().show()

+------------------------+-----+
|stalk-surface-above-ring|count|
+------------------------+-----+
|                       f|  552|
|                       k| 2372|
|                       y|   24|
|                       s| 5176|
+------------------------+-----+



In [17]:
train.groupBy('stalk-surface-below-ring').count().show()

+------------------------+-----+
|stalk-surface-below-ring|count|
+------------------------+-----+
|                       f|  600|
|                       k| 2304|
|                       y|  284|
|                       s| 4936|
+------------------------+-----+



In [18]:
train.groupBy('stalk-color-above-ring').count().show()

+----------------------+-----+
|stalk-color-above-ring|count|
+----------------------+-----+
|                     g|  576|
|                     n|  448|
|                     e|   96|
|                     o|  192|
|                     p| 1872|
|                     w| 4464|
|                     y|    8|
|                     c|   36|
|                     b|  432|
+----------------------+-----+



In [19]:
train.groupBy('stalk-color-below-ring').count().show()

+----------------------+-----+
|stalk-color-below-ring|count|
+----------------------+-----+
|                     g|  576|
|                     n|  512|
|                     e|   96|
|                     o|  192|
|                     p| 1872|
|                     w| 4384|
|                     y|   24|
|                     c|   36|
|                     b|  432|
+----------------------+-----+



In [20]:
train.groupBy('veil-color').count().show()

+----------+-----+
|veil-color|count|
+----------+-----+
|         n|   96|
|         o|   96|
|         w| 7924|
|         y|    8|
+----------+-----+



In [21]:
train.groupBy('veil-type').count().show()

+---------+-----+
|veil-type|count|
+---------+-----+
|        p| 8124|
+---------+-----+



#### Since this feature adds no value it will not be used in our model

In [22]:
train.groupBy('ring-number').count().show()

+-----------+-----+
|ring-number|count|
+-----------+-----+
|          n|   36|
|          o| 7488|
|          t|  600|
+-----------+-----+



In [23]:
train.groupBy('ring-type').count().show()

+---------+-----+
|ring-type|count|
+---------+-----+
|        l| 1296|
|        f|   48|
|        n|   36|
|        e| 2776|
|        p| 3968|
+---------+-----+



In [24]:
train.groupBy('spore-print-color').count().show()

+-----------------+-----+
|spore-print-color|count|
+-----------------+-----+
|                n| 1968|
|                k| 1872|
|                o|   48|
|                h| 1632|
|                w| 2388|
|                y|   48|
|                u|   48|
|                b|   48|
|                r|   72|
+-----------------+-----+



In [25]:
train.groupBy('population').count().show()

+----------+-----+
|population|count|
+----------+-----+
|         n|  400|
|         v| 4040|
|         y| 1712|
|         c|  340|
|         a|  384|
|         s| 1248|
+----------+-----+



In [26]:
train.groupBy('habitat').count().show()

+-------+-----+
|habitat|count|
+-------+-----+
|      l|  832|
|      g| 2148|
|      m|  292|
|      p| 1144|
|      d| 3148|
|      w|  192|
|      u|  368|
+-------+-----+



### We will remove 'veil-type' and 'stalk-root'

In [27]:
train = train.select('class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat')
cols = train.columns
train.printSchema()

root
 |-- class: string (nullable = true)
 |-- cap-shape: string (nullable = true)
 |-- cap-surface: string (nullable = true)
 |-- cap-color: string (nullable = true)
 |-- bruises: string (nullable = true)
 |-- odor: string (nullable = true)
 |-- gill-attachment: string (nullable = true)
 |-- gill-spacing: string (nullable = true)
 |-- gill-size: string (nullable = true)
 |-- gill-color: string (nullable = true)
 |-- stalk-shape: string (nullable = true)
 |-- stalk-surface-above-ring: string (nullable = true)
 |-- stalk-surface-below-ring: string (nullable = true)
 |-- stalk-color-above-ring: string (nullable = true)
 |-- stalk-color-below-ring: string (nullable = true)
 |-- veil-color: string (nullable = true)
 |-- ring-number: string (nullable = true)
 |-- ring-type: string (nullable = true)
 |-- spore-print-color: string (nullable = true)
 |-- population: string (nullable = true)
 |-- habitat: string (nullable = true)



In [29]:
# we will look at the first 5 rows to see if data is still ok and confirm the columns were removed
import pandas as pd
pd.DataFrame(train.take(5), columns=train.columns).transpose()

,0,1,2,3,4
class,p,e,e,p,e
cap-shape,x,x,b,x,x
cap-surface,s,s,s,y,s
cap-color,n,y,w,w,g
bruises,t,t,t,t,f
odor,p,a,l,p,n
gill-attachment,f,f,f,f,f
gill-spacing,c,c,c,c,w
gill-size,n,b,b,n,b
gill-color,k,k,n,n,k


### This part is where the encoding takes place. (Converting labels to numbers)

In [30]:
from pyspark.ml.feature import (VectorAssembler,OneHotEncoder,StringIndexer)

In [31]:
cap_shape_indexer = StringIndexer(inputCol='cap-shape',outputCol='cap_shapeIndex')
cap_shape_encoder = OneHotEncoder(inputCol='cap_shapeIndex',outputCol='cap_shapeVec')

In [32]:
cap_surface_indexer = StringIndexer(inputCol='cap-surface',outputCol='cap_surfaceIndex')
cap_surface_encoder = OneHotEncoder(inputCol='cap_surfaceIndex',outputCol='cap_surfaceVec')

In [33]:
cap_color_indexer = StringIndexer(inputCol='cap-color',outputCol='cap_colorIndex')
cap_color_encoder = OneHotEncoder(inputCol='cap_colorIndex',outputCol='cap_colorVec')

In [34]:
bruises_indexer = StringIndexer(inputCol='bruises',outputCol='bruisesIndex')
bruises_encoder = OneHotEncoder(inputCol='bruisesIndex',outputCol='bruisesVec')

In [35]:
odor_indexer = StringIndexer(inputCol='odor',outputCol='odorIndex')
odor_encoder = OneHotEncoder(inputCol='odorIndex',outputCol='odorVec')

In [36]:
gill_attachment_indexer = StringIndexer(inputCol='gill-attachment',outputCol='gill_attachmentIndex')
gill_attachment_encoder = OneHotEncoder(inputCol='gill_attachmentIndex',outputCol='gill_attachmentVec')

In [37]:
gill_spacing_indexer = StringIndexer(inputCol='gill-spacing',outputCol='gill_spacingIndex')
gill_spacing_encoder = OneHotEncoder(inputCol='gill_spacingIndex',outputCol='gill_spacingVec')

In [38]:
gill_size_indexer = StringIndexer(inputCol='gill-size',outputCol='gill_sizeIndex')
gill_size_encoder = OneHotEncoder(inputCol='gill_sizeIndex',outputCol='gill_sizeVec')

In [39]:
gill_color_indexer = StringIndexer(inputCol='gill-color',outputCol='gill_colorIndex')
gill_color_encoder = OneHotEncoder(inputCol='gill_colorIndex',outputCol='gill_colorVec')

In [40]:
stalk_shape_indexer = StringIndexer(inputCol='stalk-shape',outputCol='stalk_shapeIndex')
stalk_shape_encoder = OneHotEncoder(inputCol='stalk_shapeIndex',outputCol='stalk_shapeVec')

In [41]:
# Excluded due to 2480 missing values 
#stalk_root_indexer = StringIndexer(inputCol='stalk-root',outputCol='stalk_rootIndex')
#stalk_root_encoder = OneHotEncoder(inputCol='stalk_rootIndex',outputCol='stalk_rootVec')

In [42]:
stalk_surface_above_ring_indexer = StringIndexer(inputCol='stalk-surface-above-ring',outputCol='stalk_surface_above_ringIndex')
stalk_surface_above_ring_encoder = OneHotEncoder(inputCol='stalk_surface_above_ringIndex',outputCol='stalk_surface_above_ringVec')

In [43]:
stalk_surface_below_ring_indexer = StringIndexer(inputCol='stalk-surface-below-ring',outputCol='stalk_surface_below_ringIndex')
stalk_surface_below_ring_encoder = OneHotEncoder(inputCol='stalk_surface_below_ringIndex',outputCol='stalk_surface_below_ringVec')

In [44]:
stalk_color_above_ring_indexer = StringIndexer(inputCol='stalk-color-above-ring',outputCol='stalk_color_above_ringIndex')
stalk_color_above_ring_encoder = OneHotEncoder(inputCol='stalk_color_above_ringIndex',outputCol='stalk_color_above_ringVec')

In [45]:
stalk_color_below_ring_indexer = StringIndexer(inputCol='stalk-color-below-ring',outputCol='stalk_color_below_ringIndex')
stalk_color_below_ring_encoder = OneHotEncoder(inputCol='stalk_color_below_ringIndex',outputCol='stalk_color_below_ringVec')

In [46]:
# Commented out since we will not be using it
#veil_type_indexer = StringIndexer(inputCol='veil-type',outputCol='veil_typeIndex')
#veil_type_encoder = OneHotEncoder(inputCol='veil_typeIndex',outputCol='veil_typeVec')

In [47]:
veil_color_indexer = StringIndexer(inputCol='veil-color',outputCol='veil_colorIndex')
veil_color_encoder = OneHotEncoder(inputCol='veil_colorIndex',outputCol='veil_colorVec')

In [48]:
ring_number_indexer = StringIndexer(inputCol='ring-number',outputCol='ring_numberIndex')
ring_number_encoder = OneHotEncoder(inputCol='ring_numberIndex',outputCol='ring_numberVec')

In [49]:
ring_type_indexer = StringIndexer(inputCol='ring-type',outputCol='ring_typeIndex')
ring_type_encoder = OneHotEncoder(inputCol='ring_typeIndex',outputCol='ring_typeVec')

In [50]:
spore_print_color_indexer = StringIndexer(inputCol='spore-print-color',outputCol='spore_print_colorIndex')
spore_print_color_encoder = OneHotEncoder(inputCol='spore_print_colorIndex',outputCol='spore_print_colorVec')

In [51]:
population_indexer = StringIndexer(inputCol='population',outputCol='populationIndex')
population_encoder = OneHotEncoder(inputCol='populationIndex',outputCol='populationVec')

In [52]:
habitat_indexer = StringIndexer(inputCol='habitat',outputCol='habitatIndex')
habitat_encoder = OneHotEncoder(inputCol='habitatIndex',outputCol='habitatVec')

In [53]:
#veil_type and stalk_root exluded
assembler = VectorAssembler(inputCols=['cap_shapeVec',
 'cap_surfaceVec',
 'cap_colorVec',
 'bruisesVec',
 'odorVec',
 'gill_attachmentVec',
 'gill_spacingVec',
 'gill_sizeVec',
 'gill_colorVec',
 'stalk_shapeVec',
 'stalk_surface_above_ringVec',
 'stalk_surface_below_ringVec',
 'stalk_color_above_ringVec',
 'stalk_color_below_ringVec',                                   
 'veil_colorVec',
 'ring_numberVec',
 'ring_typeVec',
 'spore_print_colorVec',
 'populationVec',                                      
 'habitatVec'],
outputCol= 'features')

### this was a different way I tried to make it work without success

In [ ]:
# from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler

# categoricalColumns = ['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']
# stages = []
# for categoricalCol in categoricalColumns:
#     stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
#     encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
#     stages += [stringIndexer, encoder]
#     label_stringIdx = StringIndexer(inputCol = 'class', outputCol = 'label')
#     assemblerInputs = [c + "classVec" for c in categoricalColumns]
# assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
# stages += [assembler]

In [ ]:
# from pyspark.ml import Pipeline
# pipeline = Pipeline(stages = stages)
# pipelineModel = pipeline.fit(train)
# train = pipelineModel.transform(train)
# selectedCols = ['label'] + cols
# train = train.select(selectedCols)
# train.printSchema()

In [54]:
from pyspark.ml.classification import DecisionTreeClassifier

In [59]:
dtc = DecisionTreeClassifier(labelCol='class',featuresCol='features')

In [60]:
# Creating the Pipline, I will remove 'veil-type' in this stage as it only has one character

from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[cap_shape_indexer, cap_surface_indexer, cap_color_indexer, bruises_indexer, 
                            odor_indexer, gill_attachment_indexer, gill_spacing_indexer, gill_size_indexer,
                            gill_color_indexer, stalk_shape_indexer, stalk_surface_above_ring_indexer,
                            stalk_surface_below_ring_indexer, stalk_color_above_ring_indexer,stalk_color_below_ring_indexer,
                            veil_color_indexer, ring_number_indexer, ring_type_indexer, 
                            spore_print_color_indexer, population_indexer, habitat_indexer,
                            cap_shape_encoder, cap_surface_encoder, cap_color_encoder,bruises_encoder, 
                            odor_encoder, gill_attachment_encoder, gill_spacing_encoder, gill_size_encoder, 
                            gill_color_encoder, stalk_shape_encoder, stalk_surface_above_ring_encoder,
                            stalk_surface_below_ring_encoder, stalk_color_above_ring_encoder, stalk_color_below_ring_encoder,
                            veil_color_encoder, ring_number_encoder, ring_type_encoder, 
                            spore_print_color_encoder, population_encoder, habitat_encoder, dtc])

In [61]:
# Splitnig the data train/test (73/23) using random split
train_d,test_data = train.randomSplit([0.77,0.23])

In [62]:
# Fitting the model by using the pipline and train_data
model = pipeline.fit(train_d)

IllegalArgumentException: 'Field "features" does not exist.\nAvailable fields: class, cap-shape, cap-surface, cap-color, bruises, odor, gill-attachment, gill-spacing, gill-size, gill-color, stalk-shape, stalk-surface-above-ring, stalk-surface-below-ring, stalk-color-above-ring, stalk-color-below-ring, veil-color, ring-number, ring-type, spore-print-color, population, habitat, cap_shapeIndex, cap_surfaceIndex, cap_colorIndex, bruisesIndex, odorIndex, gill_attachmentIndex, gill_spacingIndex, gill_sizeIndex, gill_colorIndex, stalk_shapeIndex, stalk_surface_above_ringIndex, stalk_surface_below_ringIndex, stalk_color_above_ringIndex, stalk_color_below_ringIndex, veil_colorIndex, ring_numberIndex, ring_typeIndex, spore_print_colorIndex, populationIndex, habitatIndex, cap_shapeVec, cap_surfaceVec, cap_colorVec, bruisesVec, odorVec, gill_attachmentVec, gill_spacingVec, gill_sizeVec, gill_colorVec, stalk_shapeVec, stalk_surface_above_ringVec, stalk_surface_below_ringVec, stalk_color_above_ringV